In [38]:
from sklearn.ensemble import IsolationForest
from fastapi import FastAPI, Request
import pandas as pd
import numpy as np
import unicodedata
import pickle
import redis
import dotenv
import json
import os
from sqlalchemy import create_engine, Table, MetaData, insert
import psycopg2
import time

In [39]:
codificaciones_path = os.path.join("", "Codificaciones.xlsx")
codificaciones = pd.read_excel(codificaciones_path, sheet_name="codificaciones")
codificaciones["Code"] = codificaciones["Code"].astype(str).str.strip()


In [40]:
codificaciones_dict = {}
for list_value in codificaciones["List"].unique():
    codificaciones_dict[list_value] = {}
    if list_value == 'response_code':
        print("x")
    for index, row in codificaciones[codificaciones["List"] == list_value].iterrows():
        codificaciones_dict[list_value][row["Value"]] = str(row["Code"]) if not pd.isna(row["Code"]) else ""


x


In [ ]:
# testing = pd.read_csv("testing.csv", dtype=str, keep_default_na=False)

# scores = []
# preds = []
# og_scores = []
# all_data = []
# for _, row in testing.iterrows():
#     data = {}
#     data["AV4"] = {}
#     data["AV4"]["debtorParticipantCode"] = str(row["debtorParticipantCode"])
#     data["AV4"]["creditorParticipantCode"] = str(row["creditorParticipantCode"])
#     data["AV4"]["creationDate"] = str(row["creationDate"])
#     data["AV4"]["creationTime"] = str(row["creationTime"])
#     data["AV4"]["debtorId"] = str(row["debtorId"])
#     data["AV4"]["creditorId"] = str(row["creditorId"])
#     data["AV4"]["transactionType"] = str(row["transactionType"])
#     data["AV4"]["creditorCCI"] = str(row["creditorCCI"])
#     data["AV4"]["channel"] = str(row["channel2"])
#     data["AV4"]["responseCode"] = str(row["responseCode"])
#     data["AV4"]["currency"] = str(row["currency2"])
#     data["AV4"]["sameCustomerFlag"] = str(row["same_customer_flag"])
#     # print(data)
#     # print("_____________________________")
#     # print("_____________________________")
#     # print("_____________________________")
#     # print("_____________________________")
#     all_data.append(data)
#     preds.append(float(row["prediction"]))
#     og_scores.append(float(row["score"]))
#     scores.append(fraud_prediction(data)["pred"])
#     # if abs(og_scores[-1] - scores[-1]) > 0:
#     #     print("HERE", data)
# print(len(all_data))
# print(len(scores))
# print(len(og_scores))
# print(len(preds))
# for i in range(len(scores)):
#     if abs(og_scores[i] - scores[i]) > 0:
#         # print(scores[i], "-", og_scores[i], abs(scores[i] - og_scores[i]) == 0)
#         print(all_data[i])



In [ ]:

# data ={
#       "AV4":{
#          "debtorParticipantCode":"0003",
#          "creditorParticipantCode":"0054",
#          "creationDate":"20241002",
#          "creationTime":"134115",
#          "debtorId":"000933841",
#          "creditorId":"00000000",
#          "transactionType":"320",
#          "creditorCCI":"05491382913004953896",
#          "channel":"",
#          "responseCode":"00",
#          "currency":"604",
#          "sameCustomerFlag":"O"
#       }
#    }
# print(fraud_prediction(data)["pred"])


In [ ]:
# api-fraud-qa

In [ ]:

data ={
      "AV4":{
         "debtorParticipantCode":"0003",
         "creditorParticipantCode":"0054",
         "creationDate":"20241002",
         "creationTime":"134115",
         "debtorId":"000933841",
         "creditorId":"00000000",
         "transactionType":"320",
         "creditorCCI":"05491382913004953896",
         "channel":"",
         "responseCode":"00",
         "currency":"604",
         "sameCustomerFlag":"O"
      }
   }
print(fraud_prediction(data)["pred"])


In [41]:
pruebas = pd.read_csv("test.csv", dtype=str, keep_default_na=False)

pruebas["debtorParticipantCode"] = pruebas["debtor_participant"].map(codificaciones_dict["participants"])
pruebas["creditorParticipantCode"] = pruebas["creditor_participant"].map(codificaciones_dict["participants"])
pruebas["creationDate"] = pruebas["creation_date"].str.replace('-', '')
pruebas["creationTime"] = pruebas["creation_time"].str.replace(':', '')
pruebas["debtorId"] = pruebas["debtor_id"].astype(str)
pruebas["creditorId"] = pruebas["creditor_id"].astype(str)
pruebas["transactionType"] = pruebas["transaction_type"].map(codificaciones_dict["transaction_type"])
pruebas["creditorCCI"] = pruebas["creditor_cci"].astype(str)
pruebas["channel2"] = pruebas["channel"].map(codificaciones_dict["channel"])
pruebas["responseCode"] = pruebas["response_code"].map(codificaciones_dict["response_code"])
pruebas["currency2"] = pruebas["currency"].map(codificaciones_dict["currency"])


In [42]:
aver = pruebas[["debtorParticipantCode","creditorParticipantCode", "creationDate", "creationTime", "debtorId","creditorId", "transactionType", "creditorCCI", "channel2", "responseCode", "currency2", "same_customer_flag", "prediction", "score"]]
aver = aver.fillna("")
aver = aver.astype(str)
aver.to_csv("testing.csv", index=False)

In [37]:
aver

,debtorParticipantCode,creditorParticipantCode,creationDate,creationTime,debtorId,creditorId,transactionType,creditorCCI,channel2,responseCode,currency2,same_customer_flag,prediction,score
0,0002,0003,20241002,133854,008384047,603533838,320,00315701338447429752,,00,604,O,0,52.34228269
1,0003,0009,20241002,133910,00839369,00839369,320,00955821820529994482,91,00,840,M,0,0.00000000
2,0018,0002,20241002,134002,00932793,02637419,320,00247519358962002624,,00,604,O,0,12.69134350
3,0054,0002,20241002,134058,00963048,00963048,320,00219110167424405453,,00,604,M,0,0.00000000
4,0018,0002,20241002,133807,00964764,71507359,320,00240317561129202743,,00,604,O,0,14.19423568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0002,0011,20241002,134125,04651092,20532351589,320,01162100020000819301,15,00,604,O,0,52.61202158
96,0011,0002,20241002,133722,04748784,04749151,320,00238510504075403838,91,00,604,O,0,20.55754858
97,0018,0002,20241002,133859,05212346,72523395,320,00219110609145908853,,00,604,O,0,19.77477131
98,0038,0002,20241002,134136,05337596,05404039,320,00239013078635600037,15,00,604,O,0,31.04753916


In [22]:
xd = pd.read_csv("testing.csv", dtype=str)

In [14]:
raw = raw.replace("nan", "0")

In [15]:
raw

"[{'pk': '3799715309_322844_320_0003_0002_20241002', 'debtorId_x': '000020594', 'creditorCCI_x': '00304601314273835502', 'creditorId': '47649026', 'creditorIdCode': '2', 'message_id': 3799715309, 'trace': '322844', 'instruction_id': '2024100216271500028115322844', 'run_id': '20241002_0198', 'creationDate': datetime.date(2024, 10, 2), 'creationTime': datetime.time(16, 27, 15), 'channel': 'WEB', 'currency': 'Soles', 'creditorParticipant': 'Interbank', 'debtorParticipant': 'BCP', 'debtorTypeOfPerson': None, 'transactionType': 'Ordinary Transfer', 'debtorIdCode': '5', 'reasonCode': 0, 'responseCode': 'Accepted', 'same_customer_flag_x': 'O', 'creditorParticipant_ratio': 0.034482758620689655, 'creditorParticipant_currency_ratio': 0.034482758620689655, 'creditorParticipant_channel_ratio': 0.034482758620689655, 'creditorParticipant_responseCode_ratio': 0.034482758620689655, 'creditorParticipant_debtorParticipant_ratio': 0.034482758620689655, 'creditorParticipant_Weekday_ratio': 0.0344827586206

In [ ]:
data.debtorId

In [ ]:
xd.responseCode

In [ ]:
for _, row in aver.iterrows():
    print(_)

In [ ]:
pruebas.columns

In [ ]:
# redis_host = "redis-fraud" "redis_host = "localhost" "
redis_port = 6379
redis_name = "transacciones"
redis_table = "clientes"
redis_password = "#CO3D053Cce3$"
print("host", redis_host, redis_port, redis_name, redis_password)
conexion = redis.StrictRedis(
    host=redis_host, port=redis_port, password=redis_password, decode_responses=True
)

In [9]:
debtor_id = '48397284'

In [ ]:
f"{redis_name}:{redis_table}:{debtor_id}"

In [ ]:
raw_transactions = conexion.zrange(f"{redis_name}:{redis_table}:{debtor_id}", 0, -1)

In [7]:
db_params = {
    'dbname': 'coemcas',
    'user': 'coemcas',
    'password': 'C03$CMa5$2099',
    'host': '10.201.4.25',
    'port': '5432'
}

In [ ]:
connection_open = psycopg2.connect(**db_params)
cursor_open = connection_open.cursor()

In [4]:
current_directory = "" #os.path.dirname(os.path.abspath(__file__))
model_name = "mvp_1_model_90_opt_new_vars_250924.pickle"
model_path = os.path.join(current_directory, model_name)
codificaciones_path = os.path.join(current_directory, "Codificaciones.xlsx")

In [9]:
# app = FastAPI()
model: IsolationForest = pickle.load(open(model_path, "rb"))
codificaciones = pd.read_excel(codificaciones_path, sheet_name="codificaciones")
codificaciones["Code"] = codificaciones["Code"].str.strip()
codificaciones_dict = {}
for list_value in codificaciones["List"].unique():
    codificaciones_dict[list_value] = {}
    for index, row in codificaciones[codificaciones["List"] == list_value].iterrows():
        codificaciones_dict[list_value][row["Code"]] = row["Value"]

In [11]:
# TODO: use these variables next time the model is trained
# variables_todo = ['creditorParticipant_ratio', 'creditorParticipant_currency_ratio',
#               'creditorParticipant_channel_ratio',
#               'creditorParticipant_responseCode_ratio',
#                 'creditorParticipant_debtorParticipant_ratio', 'creditorParticipant_Weekday_ratio', 'creditorParticipant_time_interval_ratio', 'creditorParticipant_creditorCCI_ratio', 'currency_ratio', 'currency_channel_ratio', 'currency_responseCode_ratio', 'currency_debtorParticipant_ratio', 'currency_Weekday_ratio', 'currency_time_interval_ratio', 'currency_creditorCCI_ratio', 'channel_ratio', 'channel_responseCode_ratio', 'channel_debtorParticipant_ratio', 'channel_Weekday_ratio', 'channel_time_interval_ratio', 'channel_creditorCCI_ratio', 'responseCode_ratio', 'responseCode_debtorParticipant_ratio', 'responseCode_Weekday_ratio', 'responseCode_time_interval_ratio', 'responseCode_creditorCCI_ratio', 'debtorParticipant_ratio', 'debtorParticipant_Weekday_ratio', 'debtorParticipant_time_interval_ratio', 'debtorParticipant_creditorCCI_ratio', 'Weekday_ratio', 'Weekday_time_interval_ratio', 'Weekday_creditorCCI_ratio', 'time_interval_ratio', 'time_interval_creditorCCI_ratio', 'creditorCCI_ratio', 'hourSin', 'hourCos', 'dayOfYearSin', 'dayOfYearCos', 'dayOfMonthSin', 'dayOfMonthCos', 'dayOfWeekSin', 'dayOfWeekCos','debtorParticipant_bcp','debtorParticipant_interbank','debtorParticipant_citibank','debtorParticipant_scotiabank','debtorParticipant_bbva','debtorParticipant_banco_de_la_nacion','debtorParticipant_comercio','debtorParticipant_banco_pichincha','debtorParticipant_banbif','debtorParticipant_crediscotia_financiera','debtorParticipant_mi_banco','debtorParticipant_gnb','debtorParticipant_banco_falabella','debtorParticipant_banco_ripley','debtorParticipant_alfin_banco_s.a.','debtorParticipant_financiera_oh','debtorParticipant_financiera_efectiva','debtorParticipant_caja_piura','debtorParticipant_caja_trujillo','debtorParticipant_caja_arequipa','debtorParticipant_caja_sullana','debtorParticipant_caja_cusco','debtorParticipant_caja_huancayo','debtorParticipant_caja_ica','debtorParticipant_invalid','creditorParticipant_bcp','creditorParticipant_interbank','creditorParticipant_citibank','creditorParticipant_scotiabank','creditorParticipant_bbva','creditorParticipant_banco_de_la_nacion','creditorParticipant_comercio','creditorParticipant_banco_pichincha','creditorParticipant_banbif','creditorParticipant_crediscotia_financiera','creditorParticipant_mi_banco','creditorParticipant_gnb','creditorParticipant_banco_falabella','creditorParticipant_banco_ripley','creditorParticipant_alfin_banco_s.a.','creditorParticipant_financiera_oh','creditorParticipant_financiera_efectiva','creditorParticipant_caja_piura','creditorParticipant_caja_trujillo','creditorParticipant_caja_arequipa','creditorParticipant_caja_sullana','creditorParticipant_caja_cusco','creditorParticipant_caja_huancayo','creditorParticipant_caja_ica','creditorParticipant_invalid', 'currency_soles', 'channel_banca_movil', 'channel_invalid', 'channel_web', 'responseCode_rejected']

variables = [
    "creditorParticipant_ratio",
    "creditorParticipant_currency_ratio",
    "creditorParticipant_channel_ratio",
    "creditorParticipant_responseCode_ratio",
    "creditorParticipant_debtorParticipant_ratio",
    "creditorParticipant_Weekday_ratio",
    "creditorParticipant_time_interval_ratio",
    "creditorParticipant_creditorCCI_ratio",
    "currency_ratio",
    "currency_channel_ratio",
    "currency_responseCode_ratio",
    "currency_debtorParticipant_ratio",
    "currency_Weekday_ratio",
    "currency_time_interval_ratio",
    "currency_creditorCCI_ratio",
    "channel_ratio",
    "channel_responseCode_ratio",
    "channel_debtorParticipant_ratio",
    "channel_Weekday_ratio",
    "channel_time_interval_ratio",
    "channel_creditorCCI_ratio",
    "responseCode_ratio",
    "responseCode_debtorParticipant_ratio",
    "responseCode_Weekday_ratio",
    "responseCode_time_interval_ratio",
    "responseCode_creditorCCI_ratio",
    "debtorParticipant_ratio",
    "debtorParticipant_Weekday_ratio",
    "debtorParticipant_time_interval_ratio",
    "debtorParticipant_creditorCCI_ratio",
    "Weekday_ratio",
    "Weekday_time_interval_ratio",
    "Weekday_creditorCCI_ratio",
    "time_interval_ratio",
    "time_interval_creditorCCI_ratio",
    "creditorCCI_ratio",
    "hourSin",
    "hourCos",
    "dayOfYearSin",
    "dayOfYearCos",
    "dayOfMonthSin",
    "dayOfMonthCos",
    "dayOfWeekSin",
    "dayOfWeekCos",
    "debtorParticipant_bcp",
    "debtorParticipant_interbank",
    "debtorParticipant_citibank",
    "debtorParticipant_scotiabank",
    "debtorParticipant_bbva",
    "debtorParticipant_banco_de_la_nacion",
    "debtorParticipant_comercio",
    "debtorParticipant_banco_pichincha",
    "debtorParticipant_banbif",
    "debtorParticipant_crediscotia_financiera",
    "debtorParticipant_mi_banco",
    "debtorParticipant_gnb",
    "debtorParticipant_banco_falabella",
    "debtorParticipant_banco_ripley",
    "debtorParticipant_alfin_banco_s.a.",
    "debtorParticipant_financiera_oh",
    "debtorParticipant_financiera_efectiva",
    "debtorParticipant_caja_piura",
    "debtorParticipant_caja_trujillo",
    "debtorParticipant_caja_arequipa",
    "debtorParticipant_caja_sullana",
    "debtorParticipant_caja_cusco",
    "debtorParticipant_caja_huancayo",
    "debtorParticipant_caja_ica",
    "debtorParticipant_invalid",
    "creditorParticipant_bcp",
    "creditorParticipant_interbank",
    "creditorParticipant_citibank",
    "creditorParticipant_scotiabank",
    "creditorParticipant_bbva",
    "creditorParticipant_banco_de_la_nacion",
    "creditorParticipant_comercio",
    "creditorParticipant_banco_pichincha",
    "creditorParticipant_banbif",
    "creditorParticipant_crediscotia_financiera",
    "creditorParticipant_mi_banco",
    "creditorParticipant_gnb",
    "creditorParticipant_banco_falabella",
    "creditorParticipant_banco_ripley",
    "creditorParticipant_alfin_banco_s.a.",
    "creditorParticipant_financiera_oh",
    "creditorParticipant_financiera_efectiva",
    "creditorParticipant_caja_piura",
    "creditorParticipant_caja_trujillo",
    "creditorParticipant_caja_arequipa",
    "creditorParticipant_caja_sullana",
    "creditorParticipant_caja_cusco",
    "creditorParticipant_caja_huancayo",
    "creditorParticipant_caja_ica",
    "creditorParticipant_invalid",
    "currency_soles",
    "channel_banca_movil",
    "channel_invalid",
    "channel_web",
    "responseCode_rejected",
]

new_columns = [
    'f1d',
    'f7d',
    'f30d',
    'f90d',
    'f1d_to_creditor',
    'f7d_to_creditor',
    'f30d_to_creditor',
    'f90d_to_creditor',
    'unique_debtors_past_1d',
    'unique_debtors_past_7d',
    'unique_debtors_past_30d',
    'unique_debtors_past_90d',
    'prop_invalid_1d',
    'prop_banca_movil_1d',
    'prop_web_1d',
    'prop_atm_1d',
    'prop_invalid_7d',
    'prop_banca_movil_7d',
    'prop_web_7d',
    'prop_atm_7d',
    'prop_invalid_30d',
    'prop_banca_movil_30d',
    'prop_web_30d',
    'prop_atm_30d',
    'prop_invalid_90d',
    'prop_banca_movil_90d',
    'prop_web_90d',
    'prop_atm_90d',
]
new_ratios = [
    "creditorParticipant_creditorId_ratio",
    "currency_creditorId_ratio",
    "channel_creditorId_ratio",
    "responseCode_creditorId_ratio",
    "debtorParticipant_creditorId_ratio",
    "Weekday_creditorId_ratio",
    "time_interval_creditorId_ratio",
    "creditorCCI_creditorId_ratio",
    "creditorId_ratio",
]
variables += new_columns + new_ratios


In [12]:
# Load environment variables from .env file
dotenv.load_dotenv()

redis_host = os.getenv("REDIS_HOST")
redis_port = os.getenv("REDIS_PORT")
redis_name = os.getenv("REDIS_NAME")
redis_table = os.getenv("REDIS_TABLE")
redis_password = os.getenv("REDIS_PASSWORD")

conexion = redis.StrictRedis(
    host=redis_host, port=redis_port, password=redis_password, decode_responses=True
)

In [13]:
redis_host

In [ ]:
def decode_av4(av4_df):
    decode_column_list = {
        "debtorParticipantCode": "participants",
        "creditorParticipantCode": "participants",
        "transactionType": "transaction_type",
        "currency": "currency",
        "channel": "channel",
        "responseCode": "response_code",
    }
    if "creditorParticipant" in av4_df.columns or "debtorParticipant" in av4_df.columns:
        av4_df = av4_df.rename(
            columns={
                "debtorParticipant": "debtorParticipantCode",
                "creditorParticipant": "creditorParticipantCode",
            }
        )
    for column, type in decode_column_list.items():
        av4_df[column] = av4_df[column].apply(
            lambda x: (
                codificaciones_dict[type][x] if x in codificaciones_dict[type] else x
            )
        )

    av4_df = av4_df.rename(
        columns={
            "debtorParticipantCode": "debtorParticipant",
            "creditorParticipantCode": "creditorParticipant",
        }
    )

    return av4_df


In [ ]:
def data_preprocessing(av4_df: pd.DataFrame) -> pd.DataFrame:
    vars_to_feature_engineer = ["creationDate", "creationTime"]  # 2
    vars_to_ohe = [
        "debtorParticipant",
        "creditorParticipant",
        "transactionType",
        "currency",
        "channel",
        "responseCode",
    ]  # 3

    # vars_to_feature_engineer
    av4_df.rename(
        columns={
            "creationDate": "creationDate_stage",
            "creationTime": "creationTime_stage",
        },
        inplace=True,
    )

    av4_df["creationDate"] = pd.to_datetime(av4_df["creationDate_stage"]).dt.strftime(
        "%Y%m%d"
    )
    av4_df["creationTime_stage"] = (
        av4_df["creationTime_stage"].astype("string").str.replace(":", "")
    )
    av4_df["creationTime"] = pd.to_datetime(av4_df["creationTime_stage"], format="%H%M%S").dt.time

    av4_df.drop(["creationDate_stage", "creationTime_stage"], axis=1, inplace=True)
    av4_df["hourSin"] = av4_df["creationTime"].apply(lambda x: np.sin(2 * np.pi * x.hour / 24.0))
    av4_df["hourCos"] = av4_df["creationTime"].apply(lambda x: np.cos(2 * np.pi * x.hour / 24.0))
    av4_df["creationDate"] = pd.to_datetime(av4_df["creationDate"])

    av4_df["dayOfYearSin"] = av4_df["creationDate"].apply(lambda x: np.sin(2 * np.pi * x.timetuple().tm_yday / 365.0))
    av4_df["dayOfYearCos"] = av4_df["creationDate"].apply(lambda x: np.cos(2 * np.pi * x.timetuple().tm_yday / 365.0))
    av4_df["dayOfMonthSin"] = av4_df["creationDate"].apply(lambda x: np.sin(2 * np.pi * x.day / 31.0))
    av4_df["dayOfMonthCos"] = av4_df["creationDate"].apply(lambda x: np.cos(2 * np.pi * x.day / 31.0))
    av4_df["dayOfWeekSin"] = av4_df["creationDate"].apply(lambda x: np.sin(2 * np.pi * x.weekday() / 7.0))
    av4_df["dayOfWeekCos"] = av4_df["creationDate"].apply(lambda x: np.cos(2 * np.pi * x.weekday() / 7.0))
    av4_df["monthSin"] = av4_df["creationDate"].apply(lambda x: np.sin(2 * np.pi * x.month / 12.0))
    av4_df["monthCos"] = av4_df["creationDate"].apply(lambda x: np.cos(2 * np.pi * x.month / 12.0))

    av4_df.drop(vars_to_feature_engineer, axis=1, inplace=True)

    # vars_to_ohe
    av4_df["currency_soles"] = av4_df["currency"].apply(lambda x: 1 if x == "Soles" else 0)
    av4_df["responseCode_rejected"] = av4_df["responseCode"].apply(lambda x: 1 if x == "Rejected" else 0)

    def quitar_tildes_y_espacios(x: str) -> str:
        x = x.lower()
        x = unicodedata.normalize("NFKD", x).encode("ASCII", "ignore").decode("utf-8")
        x = x.replace(" ", "_")
        return x

    av4_df[f'debtorParticipant_{quitar_tildes_y_espacios(av4_df["debtorParticipant"].unique()[0])}'] = 1
    av4_df[f'creditorParticipant_{quitar_tildes_y_espacios(av4_df["creditorParticipant"].unique()[0])}'] = 1

    av4_df.drop(vars_to_ohe, axis=1, inplace=True)

    return av4_df


In [ ]:
def categorize_hour(hour):
    # Zero-padding if needed
    hour = str(hour).zfill(6)
    # Extract the hour part
    hour = int(hour[:2])
    # Define time interval categories
    if hour >= 0 and hour < 3:
        return "00 to 03"
    elif hour >= 3 and hour < 6:
        return "03 to 06"
    elif hour >= 6 and hour < 9:
        return "06 to 09"
    elif hour >= 9 and hour < 12:
        return "09 to 12"
    elif hour >= 12 and hour < 15:
        return "12 to 15"
    elif hour >= 15 and hour < 18:
        return "15 to 18"
    elif hour >= 18 and hour < 21:
        return "18 to 21"
    else:  # hour >= 21 or hour < 24
        return "21 to 00"


def generate_combinations(input_list):
    output_list = []

    for i in range(len(input_list)):
        # Add each element in a list of 1 item
        output_list.append([input_list[i]])

        for j in range(i + 1, len(input_list)):
            # Add combinations of two elements
            output_list.append([input_list[i], input_list[j]])

    return output_list

In [ ]:
def Ratio(dd1, output_list, lista):  # revisar la fecha
    creditor = ["debtorId"]

    dd2 = (
        dd1[lista]
        .sort_values(by=["debtorId", "creationDate", "creationTime"])
        .dropna()
        .copy()
    )
    dd2["count_cci"] = len(dd2)

    for i in output_list:
        dd2["_".join(map(str, i)) + "_cumcount"] = dd2.groupby(creditor + i).cumcount() + 1
        dd2["_".join(map(str, i)) + "_ratio"] = dd2[
            "_".join(map(str, i)) + "_cumcount"
        ] / len(dd2)

    columns_like_ratio = [col for col in dd2.columns if "ratio" in col]
    columns_like_ratio.append("target")

    print(f"DD2 antes: {len(dd2)}")

    dd2 = dd2[columns_like_ratio]
    dd2 = dd2[dd2["target"] == 1]

    print(f"DD2 despues: {len(dd2)}")

    return dd2.drop("target", axis=1)

In [ ]:
# def get_history(debtor_id):
#     hcols = [
#         "debtor_participant_code",
#         "creditor_participant_code",
#         "creation_date",
#         "creation_time",
#         "trace",
#         "debtor_id",
#         "debtor_id_code",
#         "transaction_type",
#         "channel",
#         "instruction_id",
#         "response_code",
#         "reason_code",
#         "creditor_cci",
#         "currency",
#         "score_datetime",
#         "creditor_id",
#         "creditor_id_code",
#         "same_customer_flag",
#         "creditor_id",
#         "creditor_id_code",
#         "reason_code",
#         "creditor_credit_card",
#     ]
#     raw_transactions = conexion.zrange(f"{redis_name}:{redis_table}:{debtor_id}", 0, -1)
#     transactions = [json.loads(transaction) for transaction in raw_transactions]
#     history = pd.DataFrame(transactions)
#     for col in hcols:
#         if col not in history.columns:
#             history[col] = None
#     history = history[hcols]
#     return history

In [ ]:
def get_history_sql(debtor_id, creation_time):
    connection = psycopg2.connect(**db_params)
    try:
        with connection.cursor() as cursor:
            sql_query = f"""
                SELECT * FROM stage_ipf_message
                WHERE debtor_id = '{debtor_id}'
                AND creation_date BETWEEN CURRENT_DATE - interval '90 days' AND CURRENT_DATE
                AND (creation_date < CURRENT_DATE OR (creation_date = CURRENT_DATE AND creation_time <= '{creation_time}'::time));
            """
            cursor.execute(sql_query)
            results = cursor.fetchall()
            column_names = [desc[0] for desc in cursor.description]
            AV_consolidado = pd.DataFrame(results, columns=column_names)
            
            #result_values = [tuple(row) for row in results]
            return AV_consolidado

    except Exception as e:
        print(f"An error occurred: {e}")
        return None
        
    finally:
        connection.close()

In [ ]:
def get_ratios_sql(debtor_id, creation_time, av4_df):
    
    history = get_history_sql(debtor_id, creation_time)
    print(history.columns.tolist())

    if "debtor_id" in history.columns:
        history.loc[:, "debtor_id"] = history["debtor_id"].astype(str)
    if "debtorId" in history.columns:
        history.loc[:, "debtorId"] = history["debtorId"].astype(str)

    print(f"Number of transactions: {len(history)}")
    combinations = generate_combinations(
        [
            "creditorParticipant",
            "currency",
            "channel",
            "responseCode",
            "debtorParticipant",
            "Weekday",
            "time_interval",
            "creditorCCI",
        ]
    )
    if history.empty:
        temp = av4_df.copy()
        for i in combinations:
            temp["_".join(map(str, i)) + "_ratio"] = 0
        columns_like_ratio = [col for col in temp.columns if "ratio" in col]
        return temp[columns_like_ratio]

    keep = [
        "debtorId",
        "creditorCCI",
        "creditorParticipant",
        "currency",
        "channel",
        "responseCode",
        "debtorParticipant",
        "creationDate",
        "creationTime",
        "time_interval",
        "Weekday",
        "target",
    ]

    cols = {
        "debtor_id": "debtorId",
        "debtor_participant_code": "debtorParticipant",
        "creditor_participant_code": "creditorParticipant",
        "creation_date": "creationDate",
        "creation_time": "creationTime",
        "transaction_type": "transactionType",
        "channel": "channel",
        "response_code": "responseCode",
        "creditor_cci": "creditorCCI",
        "currency": "currency",
    }

    for key in cols.keys():
        if key not in history.columns:
            history[key] = None
    history = history[cols.keys()]
    history.rename(columns=cols, inplace=True)

    history = decode_av4(history)
    history = history[history["transactionType"] == "Ordinary Transfer"]

    print(f"Ordinary Transfers: {len(history)}")

    history["target"] = 0
    temp = av4_df.copy()
    temp["target"] = 1
    #temp only has one row (transaction we want to predict)
    history = pd.concat([history, temp], axis=0, ignore_index=True)
    print(f"After concat: {len(history)}")
    history["creationDate"] = pd.to_datetime(history["creationDate"])
    history["Weekday"] = (
        history["creationDate"].apply(lambda x: x.weekday()).astype(object)
    )
    history["time_interval"] = history["creationTime"].apply(categorize_hour)

    return Ratio(history, combinations, keep)

In [ ]:
# #REDIS
# def get_ratios(debtor_id, av4_df):
#     history = get_history(debtor_id)
#     #print(history.columns.tolist())

#     if "debtor_id" in history.columns:
#         history.loc[:, "debtor_id"] = history["debtor_id"].astype(str)
#     if "debtorId" in history.columns:
#         history.loc[:, "debtorId"] = history["debtorId"].astype(str)

#     #print(f"Number of transactions: {len(history)}")
#     combinations = generate_combinations(
#         [
#             "creditorParticipant",
#             "currency",
#             "channel",
#             "responseCode",
#             "debtorParticipant",
#             "Weekday",
#             "time_interval",
#             "creditorCCI",
#         ]
#     )
#     if history.empty:
#         temp = av4_df.copy()
#         for i in combinations:
#             temp["_".join(map(str, i)) + "_ratio"] = 0
#         columns_like_ratio = [col for col in temp.columns if "ratio" in col]
#         return temp[columns_like_ratio]

#     keep = [
#         "debtorId",
#         "creditorCCI",
#         "creditorParticipant",
#         "currency",
#         "channel",
#         "responseCode",
#         "debtorParticipant",
#         "creationDate",
#         "creationTime",
#         "time_interval",
#         "Weekday",
#         "target",
#     ]

#     cols = {
#         "debtor_id": "debtorId",
#         "debtor_participant_code": "debtorParticipant",
#         "creditor_participant_code": "creditorParticipant",
#         "creation_date": "creationDate",
#         "creation_time": "creationTime",
#         "transaction_type": "transactionType",
#         "channel": "channel",
#         "response_code": "responseCode",
#         "creditor_cci": "creditorCCI",
#         "currency": "currency",
#     }

#     for key in cols.keys():
#         if key not in history.columns:
#             history[key] = None
#     history = history[cols.keys()]
#     history.rename(columns=cols, inplace=True)

#     history = decode_av4(history)
#     history = history[history["transactionType"] == "Ordinary Transfer"]

#     #print(f"Ordinary Transfers: {len(history)}")

#     history["target"] = 0
#     temp = av4_df.copy()
#     temp["target"] = 1
#     history = pd.concat([history, temp], axis=0, ignore_index=True)
#     #print(f"After concat: {len(history)}")
#     history["creationDate"] = pd.to_datetime(history["creationDate"])
#     history["Weekday"] = (
#         history["creationDate"].apply(lambda x: x.weekday()).astype(object)
#     )
#     history["time_interval"] = history["creationTime"].apply(categorize_hour)

#     return Ratio(history, combinations, keep)

In [ ]:
# #@app.post("/fraud")
# # async def fraud_prediction(request: Request):
# async def fraud_prediction():
#     start_time = time.time()
#     body = await request.json()

#     data = body["AV4"]
#     av4_df = pd.DataFrame(data, index=[0])

#     av4_df = decode_av4(av4_df)

#     ratios = get_ratios(data["debtorId"], av4_df)

#     end_time = time.time()
#     execution_time = end_time - start_time
#     print(f"El tiempo de ejecución del redis es: {execution_time} segundos")

#     av4_df = data_preprocessing(av4_df)

#     av4_df.reset_index(drop=True, inplace=True)
#     ratios.reset_index(drop=True, inplace=True)

#     av4_df = av4_df.join(ratios)

#     # print(f"Columns: {av4_df.columns}")

#     for v in variables:
#         if v not in av4_df.columns:
#             av4_df[v] = 0
    
#     time_corte_2 = time.time()
#     execution_time = time_corte_2 - end_time
#     print(f"El tiempo de procesamiento de datos es: {execution_time} segundos")

#     av4_df.fillna(-1, inplace=True)
#     prediction = model.decision_function(av4_df.drop(columns=["debtorId"])[variables])[
#         0
#     ]

#     # Normalize prediction to a value between 0 and 99
#     # TODO: check if this min_value and max_value are correct
#     min_value = -0.11774102106942208
#     max_value = 0.1602303765911105
#     # TODO might need to calculate the min and max reference scale values from training? idk if it is a good practice
#     # one alternative would be to use clipping
#     # train_anomaly_scores = clf.decision_function(X_train)
#     # train_min_score = np.min(train_anomaly_scores)
#     # train_max_score = np.max(train_anomaly_scores)

#     # 0 means it has the lowest probability of being normal, and 99 means it has the highest probability of being normal
#     normalized_prediction = (prediction - min_value) / (max_value - min_value) * 99
#     # we clip the normalized_prediction if it falls out of the range
#     normalized_prediction = max(0, min(normalized_prediction, 99))

#     time_corte_3 = time.time()
#     execution_time = time_corte_3 - time_corte_2
#     print(f"El tiempo de predicción de datos es: {execution_time} segundos")

#     return {"data": av4_df.to_dict(orient="records")[0], "pred": normalized_prediction}


In [ ]:
#@app.post("/fraud_sql")
# async def fraud_prediction_sql(request: Request):
def fraud_prediction_sql(json_data):
   start_time = time.time()
   # body = await request.json()
   body = json_data
   data = body["AV4"]
   av4_df = pd.DataFrame(data, index=[0])
   av4_df = decode_av4(av4_df)
   creation_time = data["creationTime"]
   creation_time = ":".join([creation_time[i:i+2] for i in range(0, len(creation_time), 2)])
   print(creation_time)
   ratios = get_ratios_sql(data["debtorId"], creation_time, av4_df)


    

   av4_df = data_preprocessing(av4_df)

   av4_df.reset_index(drop=True, inplace=True)
   ratios.reset_index(drop=True, inplace=True)

   av4_df = av4_df.join(ratios)

   # print(f"Columns: {av4_df.columns}")

   for v in variables:
      if v not in av4_df.columns:
         av4_df[v] = 0

   av4_df.fillna(-1, inplace=True)
   prediction = model.decision_function(av4_df.drop(columns=["debtorId"])[variables])[
      0
   ]
   # prediction2 = model.predict(av4_df.drop(columns=["debtorId"])[variables])[
   #    0
   # ]

   # Normalize prediction to a value between 0 and 99
   # TODO: check if this min_value and max_value are correct
   min_value = -0.11774102106942208
   max_value = 0.1602303765911105
   # TODO might need to calculate the min and max reference scale values from training? idk if it is a good practice
   # one alternative would be to use clipping
   # train_anomaly_scores = clf.decision_function(X_train)
   # train_min_score = np.min(train_anomaly_scores)
   # train_max_score = np.max(train_anomaly_scores)

   # return prediction #, prediction2
   # 0 means it has the lowest probability of being normal, and 99 means it has the highest probability of being normal
   normalized_prediction = (max_value - prediction) / (max_value - min_value) * 99
   # we clip the normalized_prediction if it falls out of the range
   normalized_prediction = max(0, min(normalized_prediction, 99))
   normalized_prediction = round(normalized_prediction, 8)

   return {"data": av4_df.to_dict(orient="records")[0], "pred": normalized_prediction}


json_data = {

   "AV4":{

      "debtorParticipantCode": "0002",

      "creditorParticipantCode": "0094",

      "creationDate": "20240828",

      "creationTime": "150600",

      "debtorId": "61084131",

      "transactionType": "320",

      "creditorCCI": "09400141000050230756",

      "channel": "00",

      "responseCode": "05",

      "currency": "604"

   }

}
df = fraud_prediction_sql(json_data)

In [ ]:
df["pred"]